In [1]:
import sys
import plotly.express as px
import pandas as pd
from pandarallel import pandarallel
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from src.tools import JsonHandler, concatenate_listings_datasets, return_cleaned_col_names, preprocess_text
from src.class_transformers import (
    GeographicTransformer,
    BathroomsTransformer,
    CreateVerificationsTransformer,
    AmenitiesTransformer,
    OfflineLocationFinder,
    PropertyTypeTransformer,
    HostLocationImputer,
    ScrapingDateTransformer
)
from src.function_transformers import (
    fun_tr_id_to_string,
    fun_tr_from_string_to_rate,
    fun_tr_transform_to_datetime,
    fun_tr_remove_dollar_sign,
)
from sklearn.utils import estimator_html_repr
from sklearn import set_config
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

set_config(transform_output="pandas")

pandarallel.initialize()
pd.options.display.float_format = "{:.0f}".format
handler = JsonHandler()
import pandas as pd
import numpy as np
from feature_engine.datetime import DatetimeSubtraction
from feature_engine.creation import RelativeFeatures
from feature_engine.encoding import OneHotEncoder, CountFrequencyEncoder, OrdinalEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures, PowerTransformer, RobustScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import QuantileRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV
import sys
from sklearn.neural_network import MLPRegressor



INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
#df_listings = concatenate_listings_datasets()
#host_locations = handler.import_from_json("data/mappings/host_locations.json")
#remap_baths = handler.import_from_json("data/mappings/baths.json")

In [ ]:
#df_listings.drop(
#    [
#        "description",
#        "neighborhood_overview",
#        "host_about",
#        "host_neighbourhood",
#        "neighbourhood",
#        "neighbourhood_group_cleansed",
#        "calendar_updated",
#        "license",
#        "listing_url",
#        "scrape_id",
#        "last_scraped",
#        "source",
#        "name",
#        "picture_url",
#        "host_url",
#        "host_name",
#        "host_thumbnail_url",
#        "host_picture_url",
#        "minimum_minimum_nights",
#        "maximum_minimum_nights",
#        "minimum_maximum_nights",
#        "maximum_maximum_nights",
#        "minimum_nights_avg_ntm",
#        "maximum_nights_avg_ntm",
#        "has_availability",
#        "availability_30",
#        "availability_60",
#        "availability_90",
#        "availability_365",
#        "calendar_last_scraped",
#        "number_of_reviews_ltm",
#        "number_of_reviews_l30d",
#        "instant_bookable",
#        "calculated_host_listings_count",
#        "calculated_host_listings_count_entire_homes",
#        "calculated_host_listings_count_private_rooms",
#        "calculated_host_listings_count_shared_rooms",
#    ],
#    axis=1,
#    inplace=True
#)

In [ ]:
#df_listings.set_index("id", inplace=True)
#
#df_nas_columns = pd.DataFrame(
#    {
#        "NAs": df_listings.isnull().sum(axis=1),
#        "Columns_with_NAs": df_listings.apply(
#            lambda x: ", ".join(x.index[x.isnull()]), axis=1
#        ),
#    }
#)
#
#more_than_7_missing = df_nas_columns.loc[df_nas_columns["NAs"] > 7, :].index.tolist()
#df_listings.drop(more_than_7_missing, inplace=True)

In [ ]:
#df_listings.head()

In [ ]:
#id_feature = ["host_id"]
#rate_feature = ["host_response_rate", "host_acceptance_rate"]
#time_feature = ["host_since", "first_review", "last_review"]
#neighbourhood_feature = ["neighbourhood_cleansed"]
#price_feature = ["price"]
#
## Amenities
#internet_pattern: str = r"\b(wifi|internet|ethernet|fibra|connection)\b"
#self_checkin_pattern: str = r"\b(self checkin|self check-in|self-checkin)\b"
#host_greeting_pattern: str = r"\b(host greeting|host greets you)\b"
#pool_pattern: str = r"\b(pool|pool view|shared pool)\b"
#oven_pattern: str = r"\b(oven)\b"
#microwave_pattern: str = r"\b(microwave|microonde)\b"
#garden_pattern: str = r"\b(garden|park|backyard)\b"
#streaming_pattern: str = r"\b(netflix|amazon|disney+|chromecast|apple tv|hbo|hbo max)\b"
#gym_pattern: str = r"\b(exercise|gym|fitness|private gym in building|shared gym|gym nearby|workout bench)\b"
#elevator_pattern: str = r"\b(elevator)\b"
#heating_pattern: str = r"\b(heating)\b"
#ac_pattern: str = r"\b(central air conditioning|ac|air conditioning)\b"
#safe_pattern: str = r"\b(safe|locker|lock|security|guard)\b"
#workspace_pattern: str = r"\b(workspace|work)\b"
#freezer_pattern: str = r"\b(freezer|refrigerator)\b"
#aid_pattern: str = r"\b(first aid kit|aid)\b"
#dishwasher_pattern: str = r"\b(dishwasher)\b"
#long_term_stays_pattern: str = r"\b(long term stays)\b"
#pets_pattern: str = r"\b(pets allowed)\b"
#bathtube_pattern: str = r"\b(bathtube)\b"
#bbq_grill_pattern: str = r"\b(bbq grill|grill|barbeque|barbeque utensils)\b"
#lake_bay_pattern: str = r"\b(lake view|bay view|harbor view|beach view)\b"
#
#set_amenities_remapper = [
#    (internet_pattern, "internet"),
#    (self_checkin_pattern, "self-checkin"),
#    (host_greeting_pattern, "host-greeting"),
#    (pool_pattern, "pool"),
#    (oven_pattern, "oven"),
#    (microwave_pattern, "microwave"),
#    (garden_pattern, "garden"),
#    (streaming_pattern, "streaming"),
#    (gym_pattern, "gym"),
#    (elevator_pattern, "elevator"),
#    (heating_pattern, "heating"),
#    (ac_pattern, "air-conditioning"),
#    (workspace_pattern, "workspace"),
#    (freezer_pattern, "freezer"),
#    (aid_pattern, "first-aid-kit"),
#    (dishwasher_pattern, "dishwasher"),
#    (long_term_stays_pattern, "long-term-stays"),
#    (pets_pattern, "pets-allowed"),
#    (bathtube_pattern, "bathtube"),
#    (bbq_grill_pattern, "bbq-grill"),
#    (lake_bay_pattern, "lake-bay-view")
#]
#
## Property type
#entire_property_pattern = r"\b(entire|tiny home)\b"
#private_room_pattern = r"\b(private room|room in serviced apartment|room in bed and breakfast|room in hotel|room in resort)\b"
#shared_room_pattern = r"\b(shared room|shared)\b"
#other_room_pattern = r"\b(entire|tiny home|private room|room in serviced apartment|room in bed and breakfast|room in hotel|room in resort|shared room|shared)\b"
#
#set_property_type_remapper = [
#    (entire_property_pattern, "entire_property"),
#    (private_room_pattern, "private_room"),
#    (shared_room_pattern, "shared_room"),
#    (other_room_pattern, "other"),
#]
#
#id_pipeline = Pipeline(steps=[("From ID to string", fun_tr_id_to_string)], verbose=True)
#
#rates_pipeline = Pipeline(
#    steps=[("Transform response rate", fun_tr_from_string_to_rate)], verbose=True
#)
#
#timestamp_pipeline = Pipeline(
#    steps=[("Transform to timestamp", fun_tr_transform_to_datetime)], verbose=True
#)
#
#price_pipeline = Pipeline(
#    steps=[("Trim price feature", fun_tr_remove_dollar_sign)], verbose=True
#)
#
## Apply to all dataset (feature engineering using other features)
#feature_creation_pipeline = Pipeline(
#    steps=[
#        ("Listing Locations", OfflineLocationFinder()),
#        ("Host Locations imputer", HostLocationImputer()),
#        (
#            "Host location",
#            GeographicTransformer(column="host_location", locations=host_locations),
#        ),
#        ("Host verifications", CreateVerificationsTransformer()),
#        ("Bathrooms", BathroomsTransformer(remap_baths)),
#        (
#            "Amenities",
#            AmenitiesTransformer(df=df_listings, remapper=set_amenities_remapper),
#        ),
#        (
#            "Property type",
#            PropertyTypeTransformer(
#                df=df_listings, remapper=set_property_type_remapper
#            ),
#        ),
#    ],
#    verbose=True,
#)
#print("Executing Feature Creation Pipeline...")
#df_listings = feature_creation_pipeline.fit_transform(df_listings)
#print("Feature Creation Pipeline completed!")


In [ ]:
#print("Executing preprocessing on features...")
#feature_preprocessor = ColumnTransformer(
#    remainder="passthrough",
#    n_jobs=-1,
#    force_int_remainder_cols=False,
#    transformers=[
#        ("Id", id_pipeline, id_feature),
#        ("Rates", rates_pipeline, rate_feature),
#        ("Price", price_pipeline, price_feature),
#        ("Timestamp", timestamp_pipeline, time_feature),
#    ],
#    verbose=True,
#)
#
#cleaned_df = feature_preprocessor.fit_transform(df_listings)
#print("Preprocessing on features completed!")
#cleaned_df.columns = return_cleaned_col_names(cleaned_df.columns)
#print("Cleaned feature names retrieved")
#
#del df_listings

In [ ]:
#cleaned_df.head()

In [ ]:
#pd.to_pickle(cleaned_df, "data/city_data/step_by_step.pkl")

In [41]:
df = pd.read_pickle("data/city_data/step_by_step.pkl")

In [42]:
df.head()

,host_id,host_response_rate,host_acceptance_rate,price,host_since,first_review,last_review,host_location,host_response_time,host_is_superhost,...,amenities_air-conditioning,amenities_workspace,amenities_freezer,amenities_first-aid-kit,amenities_dishwasher,amenities_long-term-stays,amenities_pets-allowed,amenities_bathtube,amenities_bbq-grill,amenities_lake-bay-view
id,,,,,,,,,,,,,,,,,,,,,
31840,380378,100,100,122,2011-02-07,2010-06-23,2024-06-08,1,within an hour,f,...,t,f,t,f,f,f,f,f,f,f
32120,99235,100,50,95,2010-03-26,2010-09-26,2023-10-09,2,within an hour,f,...,t,f,f,f,f,f,f,f,f,f
32180,13925330,100,67,109,2014-04-05,2010-06-17,2023-12-06,1,within a day,t,...,f,f,t,f,t,t,f,f,f,f
39115,167739,100,62,104,2010-07-15,2010-09-07,2024-05-15,1,within a few hours,f,...,t,t,f,f,f,f,f,f,f,f
39165,167739,100,62,121,2010-07-15,2010-12-17,2024-04-08,1,within a few hours,f,...,t,t,t,f,f,f,f,f,f,f


## Tryouts

### Predict only with numerical

In [43]:
numerical_features = [
    'host_response_rate',
    'host_acceptance_rate',
    'price',
    'host_location',
    'host_listings_count',
    'host_total_listings_count',
    'latitude',
    'longitude',
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'minimum_nights',
    'maximum_nights',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month',
    'listing_city_pop',
]

In [44]:
binary_features = [
    'host_has_profile_pic',
    'host_identity_verified',
    "host_is_superhost",
    'email_verification',
    'phone_verification',
    'work_email_verification',
    'amenities_internet',
    'amenities_self-checkin',
    'amenities_host-greeting',
    'amenities_pool',
    'amenities_oven',
    'amenities_microwave',
    'amenities_garden',
    'amenities_streaming',
    'amenities_gym',
    'amenities_elevator',
    'amenities_heating',
    'amenities_air-conditioning',
    'amenities_workspace',
    'amenities_freezer',
    'amenities_first-aid-kit',
    'amenities_dishwasher',
    'amenities_long-term-stays',
    'amenities_pets-allowed',
    'amenities_bathtube',
    'amenities_bbq-grill',
    'amenities_lake-bay-view'
]

In [45]:
other_features = list(set(df.columns.tolist()) - set(numerical_features) - set(binary_features))

## Correlation between features

In [46]:
df_corr = df.copy()
corr_full = df_corr[numerical_features].corr()
colors = ['green' if val < 0.7 else 'red' for val in corr_full.values.flatten()]
mask = np.where(corr_full < 0.7, 0, 1)

fig = px.imshow(corr_full,
                text_auto=True,
                aspect="auto",
                width=900,
                height=900
                )

fig.show()

In [47]:
fig = px.imshow(corr_full,
                text_auto=True,
                aspect="auto",
                width=900,
                height=900,
                color_continuous_scale=["green", "red"])

fig.update_traces(z=mask, colorscale=[[0, "green"], [1, "red"]])

fig.show()

### Create some features

In [48]:
df["beds_for_bedroom"] = df["beds"] / df["bedrooms"]

### Drop highly correlates

In [58]:
numerical_features = [
    "beds_for_bedroom",
    'host_response_rate',
    'host_acceptance_rate',
    'price',
    'host_location',
    'host_listings_count',
    'latitude',
    'longitude',
    'accommodates',
    'bathrooms',
    'minimum_nights',
    'maximum_nights',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_checkin',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month',
    'listing_city_pop',
]

In [59]:
df_corr = df.copy()
corr_full = df_corr[numerical_features].corr()

fig = px.imshow(corr_full,
                text_auto=True,
                aspect="auto",
                width=900,
                height=900
                )

fig.show()

In [54]:
other_features = list(set(df.columns.tolist()) - set(numerical_features) - set(binary_features))

### Exploration

*Data exploration of features. Firstly the binary ones with respect to price*.

*Then engineer the other features as needed and observe their relationship with price*.

In [ ]:
#numerical_df.replace({"t": 1, "f": 0}, inplace=True)

In [ ]:
#numerical_df.dropna(inplace=True)

In [ ]:
X = df.drop(["price"], axis=1, inplace=False)
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=874631
)

In [ ]:
wizard_pipe = Pipeline(
    steps=[
        (
            "Standardize",
            StandardScaler(),
        ),
        #(
        #    "SupportVectorRegression",
        #    SVR(
        #        kernel="rbf",
        #        gamma="auto",
        #        tol=1e-3,
        #        epsilon=0.1,
        #        verbose=True
        #    ),
        #),
        (
            "Model",
            RandomForestRegressor(
                n_estimators=100,
                criterion="squared_error",
                bootstrap=True,
                max_samples=0.8,
                oob_score=True,
                n_jobs=-1,
                random_state=874631,
                verbose=True
            ),
        ),
        #(
        #    "MLPRegressor",
        #    MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100),
        #                 activation="logistic",
        #                 solver="adam",
        #                 alpha=0.001,
        #                 batch_size=5000,
        #                 learning_rate="adaptive",
        #                 #learning_rate_init=0.01,
        #                 max_iter=2000,
        #                 shuffle=True,
        #                 random_state=874631,
        #                 tol=1e-4,
        #                 verbose=True,
        #                 warm_start=True,
        #                 momentum=0.9,
        #                 early_stopping=False,
        #                 validation_fraction=0.2,
        #    )
        #)
    ],
    verbose=True
)

In [ ]:
wizard_pipe.fit(X_train, y_train)

In [ ]:
pred = wizard_pipe.predict(X_test)
print(
    f"\nExplained variance score is {explained_variance_score(y_true=y_test, y_pred=pred)}",
    f"\nMean Absolute Error is {mean_absolute_error(y_true=y_test, y_pred=pred)}",
    f"\nMean Squared Error is {mean_squared_error(y_true=y_test, y_pred=pred)}",
    f"\nR^2 Error is {r2_score(y_true=y_test, y_pred=pred)}",
)

In [ ]:
#plot_loss = pd.DataFrame(wizard_pipe["MLPRegressor"].loss_curve_, columns=["loss"])
#fig = px.line(plot_loss, x = plot_loss.index, y = "loss")
#fig.show()